**Task on estimation of parameters for Gamma distribution (done by Vladimir Manaev).**
24th of October 2021.


In [1]:
# load important libraries and data
import numpy as np
import pickle
import scipy
X = np.array([20.5, 31.5, 47.7, 26.2, 44.0, 8.28, 30.8, 17.2, 19.9, 9.96, 55.8, 25.2, 29.0, 85.5, 15.1, 28.5, 21.4, 17.7, 6.42, 84.9])
n = len(X)
X_mean = X.mean()
X_sum = X.sum()

#Part 1: MLE estimation

To estimate maximum likelihood parameters we have to do the following steps:
1. Write the likelihood function
2. Take a natural log
3. MLE estimates maximize the likelihood function, so we can take F.O.Cs
4. First order conditions in our case will be represented by the folloing system:

$\psi$($\alpha$)=ln($\beta$)+(1/n)*$\sum_{i=1}^{n} ln(X_{i})$

$\alpha$=$\beta$* (1/n)*$\sum_{i=1}^{n} X_{i}$

where $\psi$($\alpha$): digamma function

Numerical solution of the system will give us the maximum likelihood estimation of the parameters: $\alpha_{MLE}$, $\beta_{MLE}$



In [ ]:
#import optimizer and digamma function
from scipy.special import digamma
from scipy.optimize import fsolve
import math

# here define the system
def equations(p):
    alpha, beta = p
    return (digamma(alpha) - np.log(beta) - 1/n * np.sum([np.log(x) for x in X]) , alpha - beta * np.mean(X))

# here check that numerical solutions actually solve each of the equation
def eq1(alpha, beta):
  return digamma(alpha) - np.log(beta) - 1/n * np.sum([np.log(x) for x in X])

def eq2(alpha, beta):
  return alpha - beta * np.mean(X)

alpha, beta =  fsolve(equations, (1, 1))

print(eq1(alpha, beta))
print(eq2(alpha, beta))
# here check what are the estimate values
print(alpha, beta)

0.0
0.0
2.410601621059194 0.07707019697740246


Once we get the estimates we need to study the asymptotic variance of the estimators.
We have theorem that states:
1. MLE estimates are consistent (tend to true value as n goes to infinity)
2. Asymptotic variance will be estimated as the inverse of Fisher information matrix estimated at MLE values and averaged over n.

Fisher information matrix will be minus expected value of Hessian of loglikelihood:

I($\alpha$,$\beta$)=-E[H]

Where H defined as:

\begin{pmatrix}
-trigamma(\alpha) & 1/\beta\\
1/\beta & \alpha/\beta^2
\end{pmatrix}

In [ ]:
tri_alpha = scipy.special.polygamma(1,alpha)

h1 = np.array(
    [[tri_alpha, - 1/beta],
     [-1/beta, alpha/beta**2]]
    )
Fisher=(1/n)*np.linalg.inv(h1)
Fisher
std_of_alpha=np.sqrt(0.51243413)
std_of_beta=np.sqrt(0.000647)
print(std_of_alpha,std_of_beta)

0.7158450460819018 0.02543619468395381


Our estimates of asymptotic MLE parameters standard deviations:


In [ ]:
print(std_of_alpha,std_of_beta)

0.7158450460819018 0.02543619468395381


#Part 2: Generalized method of moments for Gamma distribution

For GMM estimation, we need to equate the given theoretical moments of Gamma distribution to their sample analogies:

E[X] = $\alpha$/$\beta$ = (1/n)*$\sum_{i=1}^{n} X_{i}$

E[X^2]=$\alpha$*($\alpha$+1)/$\beta$^2=(1/n)* $\sum_{i=1}^{n} X_{i}^2$

E[1/X]=$\beta$/($\alpha$-1) = (1/n)*$\sum_{i=1}^{n} 1/X_{i}$

E[lnX]=$\psi$($\alpha$)-ln($\beta$)=(1/n)*$\sum_{i=1}^{n} lnX_{i}$

One can observe that the system is overidentified: four equations versus two unknowns.

So, the traditional method of moments (MM) will not work here. For traditional MM we have two choose 2 equations out of 4.

Generalized method of moments might be good alternative here, if we specify coherent penalty matrix.

The idea of GMM with equal (unity) costs that we optimize cost function with respect to two parameters: alpha and beta.

In other words, we choose alpha and beta in such way, so the total squared distance is minimal.

Squared_cost_fun($\alpha$,$\beta$)=(m1-$\alpha$/$\beta$)^2+(m2-$\alpha$*($\alpha$+1)/$\beta$^2)^2+(m3-$\beta$/($\alpha$-1))^2+(m4-$\psi$($\alpha$)+ln($\beta$))^2

where m1,m2,m3,m4 are sample estimates of the moments.

So, the solution to the minimization of cost function will give us:
 $\alpha_{GMM}$, $\beta_{GMM}$




In [ ]:
# Estimation of sample moments
m1 = X.mean()
m2 = np.sum([x*x for x in X])/n
m3 = np.sum([1/x for x in X])/n
m4 = np.sum([np.log(x) for x in X])/n

In [ ]:
#Define squared cost function
def fun(x0):
  alpha_m = x0[0]
  beta_m = x0[1]
  g1 = m1 - alpha_m/beta_m
  g2 = m2 - alpha_m*(alpha_m + 1)/beta_m**2
  g3 = m3 - beta_m/(alpha_m - 1)
  g4 = m4 - digamma(alpha_m) + np.log(beta_m)
  return g1 ** 2 +  g2 ** 2 +   g3 **2+ g4 **2

In [ ]:
#Optimize the function
import numpy as np
from scipy.optimize import minimize
x0 = [1,1]
res = minimize(fun, x0, method='nelder-mead',
               options={'xatol': 1e-8, 'disp': True})
alpha_m, beta_m = res.x
alpha_m, beta_m

Optimization terminated successfully.
         Current function value: 0.001803
         Iterations: 115
         Function evaluations: 223


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in double_scalars
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in log
  import sys


(2.0582996665314375, 0.0657988790734718)

Theorem of consistency of GMM estimators states that our estimates will be consistent as well under some mild technical conditions. 

So, GMM with unity penalties provides us the following estimates:


In [ ]:
print(alpha_m,beta_m)

2.0582996665314375 0.0657988790734718


One can easily check that the optimization above is equivalent to optimization of quadratic form:
gWg_transpose

where 

g is the vector of non-squared cost functions

W=diag(1) square matrix.

If we want to achieve efficiency here, we need to pick the weight matrix in a "smart" way, so it is proportional to inverse of variances.